In [29]:
# train = read.csv('./Stat441_KaggleProject/Train_for_adam.csv')
# test = read.csv('./Stat441_KaggleProject/Test_for_adam.csv')
# train_drop_col = colnames(train)[!colnames(train) %in% colnames(test)]
# train <- train[, !names(train) %in% train_drop_col]
# CLASSes = grep("^CLASS.", colnames(train))
# COMPLEXes = grep("^COMPLEX.", colnames(train))
# PHENOTYPEs = grep("^PHENOTYPE.", colnames(train))
# MOTIFs = grep("^MOTIF.", colnames(train))
# NUMINTERs = grep("^NUM.INTERACING.", colnames(train))
# colnames(test)[!colnames(test) %in% colnames(train)]

In [209]:
train = read.csv('./Stat441_KaggleProject/train_selected_adam.csv', header=TRUE)
test = read.csv('./Stat441_KaggleProject/test_selected_adam.csv', header=TRUE)

train$essential[train$essential == 0] <- 1
test$essential[test$essential == 0] <- 3
test$essential[test$essential == 1] <- 2
test$essential[test$essential == 2] <- 1
test$essential[test$essential == 3] <- 2

cont_cols = c(sapply(seq(45, 57), function(x) {paste("X4", toString(x), sep='')}), 'correlation', 'protein')
# binary_cols = c('X94','X95','X96','X97','X298', 'X307', 'X342')
# num_cols = c(cont_cols, binary_cols)
num_cols = c(cont_cols)
train[, !names(train) %in% num_cols] <- lapply(train[, !names(train) %in% num_cols], factor)
# test$chromosome[!test$chromosome %in% levels(train$chromosome)] <- '17'
test[, !names(test) %in% num_cols] <- lapply(test[, !names(test) %in% num_cols], factor)
train$chromosome <- factor(train$chromosome, levels = levels(test$chromosome))
train$chromosome[is.na(train$chromosome)] <- '3'
# train$binarysum <- rowSums(train[binary_cols])
# train <- train[, !names(train) %in% binary_cols]
# test$binarysum <- rowSums(test[binary_cols])
# test <- test[, !names(test) %in% binary_cols]

train_size <- floor(0.7 * nrow(train))
train_ind <- sample(seq_len(nrow(train)), size = train_size, replace = FALSE)
valid_ind <- seq_len(nrow(train))[-train_ind]
# train_ind <- sample(seq_len(length(train_ind)), size = floor(1.5 * train_size), replace = TRUE)
train_df <- train[train_ind, ]
valid_df <- train[valid_ind, ]
train_df$label <- factor(train_df$label)

In [210]:
levels(test$chromosome)

[1] "0"  "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14"
[16] "15"

In [211]:
# library(e1071)
# model <- svm(label ~ . - protein, data = train_df, kernel="linear", cost = 10, epsilon = 0.1)
# valid_preds <- predict(model, valid_df)
# valid_preds <- factor(valid_preds, levels=levels(valid_df$label))
# valid_l <- valid_preds == valid_df$label
# sum(valid_l) / length(valid_l)
# table(valid_preds) / length(valid_preds)

[1] 0.7385787

valid_preds
          0           1           2           3           4           5 
0.456852792 0.230964467 0.068527919 0.076142132 0.032994924 0.030456853 
          6           7           8           9          10          11 
0.030456853 0.022842640 0.012690355 0.012690355 0.017766497 0.000000000 
         12          13          14 
0.002538071 0.002538071 0.002538071 

In [219]:
C <- 10 * seq(0.1, 1, 0.1)
epsilon <- seq(0.1, 1, 0.1)
g <- seq(0.1, 1, 0.1)
max_acc <- 0
max_C <- C[1]
max_epsilon <- epsilon[1]
max_g <- g[1]
max_pred <- c()
for (cost in C){
  for (eps in epsilon){
    model <- svm(label ~ . - protein, data = train_df, kernel="linear", cost = cost, epsilon = eps, gamma=g)
    predicted <- predict(model, valid_df)
    predicted <- factor(predicted, levels=levels(valid_df$label))
    accu_l <- predicted == valid_df$label
    accuracy <- sum(accu_l) / length(accu_l)
    if (accuracy > max_acc){
      max_acc <- accuracy
      max_C <- cost
      max_epsilon <- eps
      max_model <- model
      max_g <- g
      max_pred <- predicted
    }
  }
}

In [220]:
table(max_pred) / length(max_pred)

max_pred
          0           1           2           3           4           5 
0.459390863 0.269035533 0.043147208 0.078680203 0.025380711 0.040609137 
          6           7           8           9          10          11 
0.032994924 0.022842640 0.015228426 0.007614213 0.005076142 0.000000000 
         12          13          14 
0.000000000 0.000000000 0.000000000 

In [221]:
for (i in colnames(test)){
    n1 = levels(test[, i])
    n2 = levels(train[, i])
    if (length(n1) != length(n2) || sum(n1 != n2)){
#         print(i)
#         print(levels(test[, i]))
#         print(levels(train[, i]))
        test[, i] <- factor(test[, i], levels = levels(train[, i]))
    }
}

test_preds <- predict(max_model, test)
test$label <- test_preds
table(test_preds) / length(test_preds)

test_preds
          0           1           2           3           4           5 
0.454976303 0.226540284 0.030331754 0.099526066 0.019905213 0.052132701 
          6           7           8           9          10          11 
0.062559242 0.018009479 0.024644550 0.008530806 0.002843602 0.000000000 
         12          13          14 
0.000000000 0.000000000 0.000000000 

In [222]:
result <- data.frame("protein" = test$protein, "label" = test$label)
result_agg <- aggregate(result,
                by = list(result$protein),
                FUN = function(x) tail(names(sort(table(x))), 1))
result_agg <- data.frame("protein" = result_agg$protein, "label" = result_agg$label)
table(result_agg$label) / length(result_agg$label)


          0           1          10           2           3           4 
0.422572178 0.304461942 0.002624672 0.057742782 0.057742782 0.020997375 
          5           6           7           8           9 
0.060367454 0.026246719 0.020997375 0.023622047 0.002624672 

In [223]:
colnames(result_agg) <- c('Key', 'Label')
write.csv(result_agg,'./result/result_Apr3_2.csv', row.names = FALSE, quote=FALSE)